In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Specify the path to your CSV file
csv_file_path = './drive/MyDrive/aist4010_project/form_records.csv'

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path, index_col=0)

# Now you can work with the DataFrame
print(df.head())  # Print the first few rows of the DataFrame

  Race_Index Place      Date    RC/Track/Course  Distance Going Race_Class  \
0        463    03  28/02/24    HV / Turf / "A"      1200     G          3   
1        422    01  15/02/24    HV / Turf / "C"      1200     G          3   
2        386    04  31/01/24    HV / Turf / "A"      1200     G          3   
3        332    03  10/01/24    HV / Turf / "B"      1200     G          3   
4        295    07  29/12/23  HV / Turf / "C+3"      1200     G          3   

  Draw Rating Trainer     Jockey    LBW Win_Odds  Actual_Weight  \
0    2     72  J Size  A Hamelin      2      6.2            128   
1   11     67  J Size   Z Purton     HD      5.2            127   
2    8     68  J Size   M F Poon  2-3/4       14            123   
3    3     68  J Size   A Atzeni  2-1/2      7.3            126   
4    7     68  J Size   Z Purton      2      4.8            125   

  Running_Position Finish_Time Declared_Horse_Weight Gear Video_Replay  \
0            8 7 3     1.10.98                  1078  

In [ ]:
# Drop multiple columns
columns_to_drop = [ 'Trainer','Jockey', 'LBW', 'Running_Position', 'Gear', 'Video_Replay', 'Video_Replay_2']
df = df.drop(columns_to_drop, axis=1)

In [ ]:
# Transform RC/Track/Course into columns
df[['RC', 'Track', 'Course']] = df['RC/Track/Course'].str.split('/', expand=True)

df['RC'] = df['RC'].str.strip()
df['Track'] = df['Track'].str.strip()
df['Course'] = df['Course'].str.strip()

df = df.drop(['RC/Track/Course'], axis=1)

print(df.head())

  Race_Index Place      Date  Distance Going Race_Class Draw Rating Win_Odds  \
0        463    03  28/02/24      1200     G          3    2     72      6.2   
1        422    01  15/02/24      1200     G          3   11     67      5.2   
2        386    04  31/01/24      1200     G          3    8     68       14   
3        332    03  10/01/24      1200     G          3    3     68      7.3   
4        295    07  29/12/23      1200     G          3    7     68      4.8   

   Actual_Weight Finish_Time Declared_Horse_Weight  RC Track Course  
0            128     1.10.98                  1078  HV  Turf    "A"  
1            127     1.10.51                  1075  HV  Turf    "C"  
2            123     1.10.18                  1083  HV  Turf    "A"  
3            126     1.10.30                  1074  HV  Turf    "B"  
4            125     1.10.54                  1095  HV  Turf  "C+3"  


In [ ]:
def parse_time(time_str):
    parts = time_str.split('.')
    try:
        if len(parts) == 3:
            return int(parts[0]), int(parts[1]), int(parts[2])  # minutes, seconds, milliseconds
        elif len(parts) == 2:
            return 0, int(parts[0]), int(parts[1])  # default minutes to 0, then seconds, milliseconds
        else:
            return None  # Return None for unexpected formats
    except ValueError:
        return None  # Return None if any part is not an integer

# Apply the function to each row and create new columns
df['parsed_time'] = df['Finish_Time'].apply(parse_time)
df[['Finish_Minute', 'Finish_Second', 'Finish_MilliSecond']] = pd.DataFrame(df['parsed_time'].tolist(), index=df.index)

In [ ]:
# Convert column to numeric values
df['Place'] = pd.to_numeric(df['Place'], errors='coerce')
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%y')
df['Win_Odds'] = pd.to_numeric(df['Win_Odds'], errors='coerce')
df['Draw'] = pd.to_numeric(df['Draw'], errors='coerce')
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')
df['Actual_Weight'] = pd.to_numeric(df['Actual_Weight'], errors='coerce')
df['Declared_Horse_Weight'] = pd.to_numeric(df['Declared_Horse_Weight'], errors='coerce')

# Remove double quotes from 'Course' column
df['Course'] = df['Course'].str.strip('"')

# Drop rows with missing or invalid entry
df = df.drop(df[df.values == '--'].index)

# # Split 'Finish_Time' column into hours, minutes, and seconds
# df[['Finish_Minute', 'Finish_Second', 'Finish_MilliSecond']] = df['Finish_Time'].str.split('.', expand=True).astype(int)

# Drop the original 'Finish_Time' column
df = df.drop('Finish_Time', axis=1)

# Convert the time components to seconds and calculate the total finishing time in milliseconds: 1.10.98
df['Finish_Time_ms'] = (df['Finish_Minute'] * 60 * 1000) + (df['Finish_Second'] * 1000) + (df['Finish_MilliSecond'] * 10)

# Drop the original 'Finish_Time' column and the time components columns
df = df.drop(['Finish_Minute', 'Finish_Second', 'Finish_MilliSecond', 'parsed_time'], axis=1)

print(df.head())

  Race_Index  Place       Date  Distance Going Race_Class  Draw  Rating  \
0        463    3.0 2024-02-28      1200     G          3   2.0    72.0   
1        422    1.0 2024-02-15      1200     G          3  11.0    67.0   
2        386    4.0 2024-01-31      1200     G          3   8.0    68.0   
3        332    3.0 2024-01-10      1200     G          3   3.0    68.0   
4        295    7.0 2023-12-29      1200     G          3   7.0    68.0   

   Win_Odds  Actual_Weight  Declared_Horse_Weight  RC Track Course  \
0       6.2            128                 1078.0  HV  Turf      A   
1       5.2            127                 1075.0  HV  Turf      C   
2      14.0            123                 1083.0  HV  Turf      A   
3       7.3            126                 1074.0  HV  Turf      B   
4       4.8            125                 1095.0  HV  Turf    C+3   

   Finish_Time_ms  
0         70980.0  
1         70510.0  
2         70180.0  
3         70300.0  
4         70540.0  


In [ ]:
print(df.head())
print(df.shape[0])

# Drop columns containing NaN values
df = df.dropna().reset_index(drop=True)

print(df.head())
print(df.shape[0])

print(df)
df.to_csv('./drive/MyDrive/aist4010_project/form_records_cleaned.csv')

  Race_Index  Place       Date  Distance Going Race_Class  Draw  Rating  \
0        463    3.0 2024-02-28      1200     G          3   2.0    72.0   
1        422    1.0 2024-02-15      1200     G          3  11.0    67.0   
2        386    4.0 2024-01-31      1200     G          3   8.0    68.0   
3        332    3.0 2024-01-10      1200     G          3   3.0    68.0   
4        295    7.0 2023-12-29      1200     G          3   7.0    68.0   

   Win_Odds  Actual_Weight  Declared_Horse_Weight  RC Track Course  \
0       6.2            128                 1078.0  HV  Turf      A   
1       5.2            127                 1075.0  HV  Turf      C   
2      14.0            123                 1083.0  HV  Turf      A   
3       7.3            126                 1074.0  HV  Turf      B   
4       4.8            125                 1095.0  HV  Turf    C+3   

   Finish_Time_ms  
0         70980.0  
1         70510.0  
2         70180.0  
3         70300.0  
4         70540.0  
14964
  